In [ ]:
# This code cell installs packages on Colab
!pip install pyomo
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

from pyomo.environ import *
from pyomo.dae import *
import numpy as np
from google.colab import files




In [ ]:
# 1: this script solves the steady-advection-diffusion-reaction equations using the pyomo.dae

# 2: numerical examples are modified from the paper: Isogeometric analysis: CAD, finite elements, NURBS,
# exact geometry and mesh refinement, T.J.R. Hughes et.al

# 3: The code structure refers to the paper: pyomo.dae: a modeling and automatic discretization
# framework for optimization with differential and algebraic equations, Nicholson et al.


# 4: This code uses special constraints to make sure the numerical solution satisfying the physical and analytical laws

# 5: This code also has time-dependent part

# space-time variables
m      = ConcreteModel()
m.t    = ContinuousSet(bounds = (0,1.5))
m.x    = ContinuousSet(bounds = (0,1))
m.y    = ContinuousSet(bounds = (0,1))
m.phi  = Var(m.t, m.x, m.y)


# define derivative variables
m.DphiDx   = DerivativeVar(m.phi, wrt = m.x)
m.DphiDy   = DerivativeVar(m.phi, wrt = m.y)
m.DphiDt   = DerivativeVar(m.phi, wrt = m.t)
m.DphiDx_2 = DerivativeVar(m.phi, wrt = (m.x, m.x))
m.DphiDy_2 = DerivativeVar(m.phi, wrt = (m.y, m.y))

# define PDE parameters
m.c 	    = Param(initialize = 1.0)             # the reaction constant
m.nu      = Param(initialize = 1e-6)            # the diffusivity
m.bx      = Param(initialize = 1.0/2)           # the advection velocity, x-component
m.by      = Param(initialize = np.sqrt(3)/2.0)  # the advection velocity, y-component


# define adr pde, i: x, j: y
def ADR_pde(m,k,i,j):
	if j == m.y.first(): # boundary condition constraint skip
	 	return Constraint.Skip
	if i == m.x.first(): # boundary condition constraint skip
	 	return Constraint.Skip
	return m.DphiDt[k,i,j] + m.bx * m.DphiDx[k,i,j] + m.by * m.DphiDy[k,i,j]- m.nu*m.DphiDx_2[k,i,j] - m.nu*m.DphiDy_2[k,i,j]   == 0.0 # PDE
m.pde = Constraint(m.t, m.x, m.y, rule = ADR_pde )   

#Bottom boundary condition [phi = 1 @ y = 0]
def BC1(m,k,i):
	if i == m.x.first():
		return Constraint.Skip
	return m.phi[k, i,m.y.first()] == 1.0
m.BCy_0 = Constraint(m.t, m.x, rule = BC1)


# Left boundary condition [phi = 1 @ 0<y<0.2, phi = 0 @ y>0.2]
def BC4(m,k,j):
	if j> 0.2:
		return m.phi[k, m.x.first(),j] == 0.0
	else:
		return m.phi[k, m.x.first(),j] == 1.0
m.BCx_0 = Constraint(m.t, m.y, rule = BC4)


#Special constraint1: Non-negativity
def Non_negativity(m,k,i,j):
  if j>np.sqrt(3)*i + 1.0/5 - 1.0/7 and j<np.sqrt(3)*i + 1.0/5 + 1.0/7: # active region
    return m.phi[k,i,j] >= 0.0
  else: 
    return Constraint.Skip
m.SP1 = Constraint(m.t, m.x, m.y, rule = Non_negativity)


# #Special constraint2: Maximum principle
def MP(m,k,i,j):
  if j>np.sqrt(3)*i + 1.0/5 - 1.0/7 and j<np.sqrt(3)*i + 1.0/5 + 1.0/7: # active region
	  return m.phi[k,i,j] <= 1.0
  else: 
    return Constraint.Skip
m.SP2 = Constraint(m.t, m.x, m.y, rule = MP)

# Montontonicity in y direction
def Monontonicity_y(m,k,i,j):
  if j>np.sqrt(3)*i + 1.0/5 - 1.0/7 and j<np.sqrt(3)*i + 1.0/5 + 1.0/7: # active region
    return m.DphiDy[k,i,j] <= 0 
  else: 
    return Constraint.Skip
m.SP3 = Constraint(m.t, m.x, m.y, rule = Monontonicity_y)

# Montontonicity in x direction
def Monontonicity_x(m,k,i,j):
  if j>np.sqrt(3)*i + 1.0/5 - 1.0/7 and j<np.sqrt(3)*i + 1.0/5 + 1.0/7: # active region
    return m.DphiDx[k,i,j] >= 0 
  else: 
    return Constraint.Skip
m.SP4 = Constraint(m.t, m.x, m.y, rule = Monontonicity_x)

# trivial obj
m.obj = Objective(expr = 1)

# discretization and solve, backward FD in time, orthogonal collocation in space
discretizer_time = TransformationFactory('dae.finite_difference')
discretizer_time.apply_to(m,nfe=15,wrt=m.t, scheme ='BACKWARD')
discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(m,nfe=10,ncp=3,wrt=m.x)
discretizer.apply_to(m,nfe=10,ncp=3,wrt=m.y)

# solve and save
solver = SolverFactory('ipopt',executable='/content/ipopt')
solver.options['max_iter']= 500 #number of iterations you wish
results = solver.solve(m, tee=True) 

# extract data
t   = np.zeros(len(m.t))
x   = np.zeros(len(m.x))
y   = np.zeros(len(m.y))
phi = np.zeros( (len(m.t), len(m.x), len(m.y) ) )

for k in range(len(m.t)):
  t[k] = value(m.t[k+1])
  savename = 'phi-t=' + str(t[k]) + '.csv'
  for i in range(len(m.x)):
    for j in range(len(m.y)):
      x[i] = value(m.x[i+1])
      y[j] = value(m.y[j+1])
      phi[k,i,j] = value(m.phi[t[k],x[i],y[j]])
  if k%5 == 0:
    np.savetxt(savename, phi[k,:,:], delimiter=',')  
  if k == 0:
    np.savetxt('x-time.csv', x, delimiter=',')   
    np.savetxt('y-time.csv', y, delimiter=',') 


Ipopt 3.12.13: max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   413821
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    90211
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds: